In [2]:
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
import cv2
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
from tensorflow.keras.utils import to_categorical


In [ ]:
model = keras.models.load_model("mcombo.h5")

In [ ]:
def prepro(gray):
    cv2_imshow(gray)
    kernel = np.ones((3,3), np.uint8)
    gray= cv2.erode(gray, kernel, iterations=1)
    # gray= cv2.dilate(gray, kernel, iterations=1)
    # cv2_imshow(gray)
    blur=cv2.GaussianBlur(gray,(5,5),2)
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cv2_imshow(thresh)
    
    print(thresh.shape)
    h=thresh.shape[0]
    w=thresh.shape[1]
    prev=0
    cont=0
    v=[]
    for i in range(0,w):
        x=0
        for j in range(0,h):
             if(thresh[j][i]==0):
                 x+=1
        if(x==0 and cont==1):
            if(i-prev<10):
               cont=0
            else:
                  cont=0
                  v.append((prev,i))
        elif(x>0 and cont==0):
              prev=i
              cont=1
    
    # for x in v:
    #     cv2.line(img,(x[0],0),(x[0],h-1),(255,0,0),3)
    #     cv2.line(img,(x[1],0),(x[1],h-1),(255,0,0),3)
    
    prev=0        
    u=[]
    for x in v:
          s=x[0]
          e=x[1]
          cont=0
          for i in range(0,h):
               z=0
               for j in range(s,e+1):
                   if(thresh[i][j]==0):
                       z+=1
               if(z>0 and cont==0):
                    prev=i
                    cont=1
               elif(z==0 and cont==1):
                    if(i-prev<10):
                         cont=0
                    else:
                      u.append((prev,i))
                      break
    print(v)
    print(u)
    v2=[]
    u2=[]
    i=0
    while i<len(v)-1:
          if(v[i+1][0]-v[i][1]<3):
               v2.append((v[i][0],v[i+1][1]))
               u2.append((min(u[i][0],u[i+1][0]),max(u[i][1],u[i+1][1])))
               i+=2
          else:
             v2.append(v[i])
             u2.append(u[i])
             i+=1
    if(i==len(v)-1):
        v2.append(v[-1])
        u2.append(u[-1])
    
    st_list=[]
    end_list=[]
    for i in range(len(v2)):
       if((v2[i][1]-v2[i][0])>30 and u2[i][1]-u2[i][0]>30):
           cv2.rectangle(img,(v2[i][0],u2[i][0]),(v2[i][1],u2[i][1]),(255,0,0),3)
           st_list.append((u2[i][0],v2[i][0]))
           end_list.append((u2[i][1],v2[i][1]))
    cv2_imshow(img)    
    return thresh,st_list,end_list
    

In [ ]:
classes = "1,2,3,4,5,6,7,A,B,d,e,g,G,Z,W,H,I,J,R,t,K,M,S,N,X,b"
index =   "1,2,3,4,5,6,7,10,11,38,39,41,16,35,32,17,18,19,27,46,20,22,28,23,33,37"
values = classes.split(',')
keys = index.split(',')
map = dict(zip(keys, values))
img = cv2.imread('test9.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img2,st,end=prepro(gray)

In [ ]:
fans=''
for i in range(len(st)):
      eximg=img2[st[i][0]:end[i][0],st[i][1]:end[i][1]]
      eximg=cv2.copyMakeBorder(eximg,40,40,40,40,cv2.BORDER_CONSTANT,value=(255,255,255))
      kernel = np.ones((5, 5), np.uint8)
      eximg2=cv2.dilate(eximg,kernel,10)
      cv2_imshow(eximg2) 
      eximg=eximg/255
      eximg=cv2.resize(eximg,(28,28))
      ans=model.predict(eximg.reshape(-1,28,28,1))
      prediction = ans.argmax(axis=1)
      conf=np.max(ans)
      print(prediction,conf)
      fans+=map[str(prediction[0])]

print(fans)